This notebook is for processing the data to prapare it for model construction

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import _pickle as pickle

%matplotlib inline

## Data

In [2]:
# Define constant
INPUT_DIR = "../input"
INPUT_DATA = os.listdir(INPUT_DIR)
FILE_NAME = [s.split(".")[0] for s in INPUT_DATA]

# load data
for file_name, data in zip(FILE_NAME, INPUT_DATA):
    exec("{}=pd.read_csv(os.path.join(INPUT_DIR,\"{}\"))".format(file_name, data))

# rename
order_products_train = order_products__train
order_products_prior = order_products__prior

## Get basket

Target: <br>
1. get each user's previous purchase info (each include the items they purchased and other information)
2. dispose of the user information not in test

Data:<br>
1. order_products__*<br>
2. orders

In [3]:
order_products__prior.head(2)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1


In [4]:
order_products__train.head(2)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1


In [5]:
orders.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


1.For each order_id, get the coresponding purchased items

In [6]:
def get_basket(order):
    col = ["order_id", "product_id"]
    return order[col].groupby(col[0])[col[1]].apply(lambda x: x.tolist())

2.Remove users in prior but not in train test eval_set

In [7]:
orders_prior = orders.ix[orders.eval_set == "prior",:]
orders_train = orders.ix[orders.eval_set == "train",:]
orders_test = orders.ix[orders.eval_set == "test",:]
orders_train_test = np.concatenate([orders_train["user_id"], orders_test["user_id"]])

train_test_user = dict.fromkeys(np.unique(orders_train_test))
prior_in_train_test = [x in train_test_user for x in orders_prior["user_id"]]

In [8]:
# check if there is any users in prior but not in train test eval_set
assert all(prior_in_train_test) == True

3.Merge the basket

In [9]:
order_prior_basket = get_basket(order_products__prior)
order_train_basket = get_basket(order_products__train)

order_prior_basket = order_prior_basket.reset_index()
order_train_basket =order_train_basket.reset_index()

In [10]:
orders_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [11]:
orders_prior_merge = pd.merge(orders_prior, order_prior_basket, on="order_id")
orders_train_merge = pd.merge(orders_train, order_train_basket, on="order_id")

In [12]:
orders_prior_merge.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id
0,2539329,1,prior,1,2,8,NaN,"[196, 14084, 12427, 26088, 26405]"
1,2398795,1,prior,2,3,7,15.0,"[196, 10258, 12427, 13176, 26088, 13032]"


In [13]:
orders_train_merge.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id
0,1187899,1,train,11,4,8,14.0,"[196, 25133, 38928, 26405, 39657, 10258, 13032..."
1,1492625,2,train,15,1,11,30.0,"[22963, 7963, 16589, 32792, 41787, 22825, 1364..."


In [14]:
del orders
del orders_prior
del orders_test
del orders_train
del orders_train_test
del order_products_train
del order_products_prior

In [15]:
orders_train_merge.to_csv("../transform/orders_train_merge.csv")

In [16]:
orders_prior_merge.to_csv("../transform/orders_prior_merge.csv")

In [ ]:
orders_test.to_csv("../transform/orders_test.csv")